# This notebook is used to conduct experiments on neural network graph extraction.
## Procedure
1. Write a script which extracts information of the graph from a neural network model using flexible transform. It should be configurable which network types should be transformed in a low level manner and how often it is executed. 
2. Get the CSV data of a module e.g. AlexNet, ResNet or GoogleNet
3. Use pandas to load the csv data as dataframe
4. Use seaborn or plotly to visualize the data with boxplots

### In the following a script is provided with these properties:
- The row names are: 
    Source, GPU, UUID, datetime_start, datetime_end, model, low_level_modules, elapsed_time  
  

In [2]:
import csv
import os
from deepstruct.node_map_strategies import AdjustableNodeMap, HighLevelNodeMap
from deepstruct.traverse_strategies import FXTraversal
from deepstruct.flexible_transform import GraphTransform
import platform
import psutil
import sys
import time

def get_model_graph_stats(model, input, rounds : int, low_level_module_maps=None):
    if low_level_module_maps is None:
        low_level_module_maps = {}
    entries = []
    sys_info = get_system_info()
    source = 'Laptop'
    model_name = model.__class__.__name__
    nodemap = AdjustableNodeMap(low_level_module_maps) 
    # nodemap.add_node_mapper(low_level_module_maps) # does not work
    graph_transformer = GraphTransform(input, traversal_strategy=FXTraversal(), node_map_strategy=nodemap)
    
    for i in range(rounds):
        start_time = time.perf_counter()
        graph_transformer.transform(model)
        end_time = time.perf_counter()
        elapsed_time = end_time - start_time
        entries.append([source, model_name, elapsed_time])
        
    directory_path = 'graph_stats'

    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        
    file_name = 'graph_data.csv'
    file_path = os.path.join(directory_path, file_name)
    with open(file_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Source', 'Model', 'Elapsed Time'])
        csvwriter.writerows(entries)
        
    
def get_system_info():
    info = {}
    info['platform'] = platform.system()
    info['platform-release'] = platform.release()
    info['platform-version'] = platform.version()
    info['architecture'] = platform.machine()
    # info['hostname'] = platform.node()
    #info['ip-address'] = socket.gethostbyname(socket.gethostname())
    # info['mac-address'] = ':'.join(re.findall('..', '%012x' % uuid.getnode()))
    info['processor'] = platform.processor()
    info['ram'] = str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
    return info


In [14]:
import torch
import torchvision

get_model_graph_stats(torchvision.models.resnet152(True), torch.rand(1, 3, 224, 224), 11)

opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 517/517 [00:00<00:00, 33140.59it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 517/517 [00:00<00:00, 33090.02it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

...

Tracing Nodes: 100%|██████████| 517/517 [00:00<?, ?it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 517/517 [00:00<00:00, 31696.54it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 517/517 [00:00<?, ?it/s]


In [4]:
import torch
import torchvision
from deepstruct.node_map_strategies import Linear2LayerMapper

get_model_graph_stats(torchvision.models.resnet18(True), torch.rand(1, 3, 224, 224), 11, low_level_module_maps={torch.nn.Linear: Linear2LayerMapper(-1)})


C:\Users\ericw\AppData\Local\pypoetry\Cache\virtualenvs\deepstruct-WwEyIJlS-py3.10\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] Die angegebene Prozedur wurde nicht gefunden
  warn(f"Failed to load image Python extension: {e}")


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 71/71 [00:02<00:00, 29.01it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 71/71 [00:02<00:00, 32.30it/s]


...

Tracing Nodes: 100%|██████████| 71/71 [00:02<00:00, 30.71it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 71/71 [00:02<00:00, 26.28it/s]


opcode         name                   target                                                          args                                   kwargs
-------------  ---------------------  --------------------------------------------------------------  -------------------------------------  --------
placeholder    x                      x                                                               ()                                     {}
call_module    conv1                  conv1                                                           (x,)                                   {}
call_module    bn1                    bn1                                                             (conv1,)                               {}
call_module    relu                   relu                                                            (bn1,)                                 {}
call_module    maxpool                maxpool                                                         (relu,)                 

Tracing Nodes: 100%|██████████| 71/71 [00:02<00:00, 26.42it/s]


### Here the visualization is made